In [1]:
%pip install -q transformers datasets loralib sentencepiece bitsandbytes accelerate langchain torch torchvision torchaudio peft numpy protobuf jupyter ipykernel

Note: you may need to restart the kernel to use updated packages.


In [2]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
import torch

base = 'kfkas/Llama-2-ko-7b-Chat'
model = AutoModelForCausalLM.from_pretrained(
    base,
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    ),
    device_map="auto"
)

local_llm = HuggingFacePipeline(pipeline=pipeline(
    "text-generation",
    model=model,
    tokenizer=base,
    max_length=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.2,
    return_full_text=False,
    # eos_token_id=2,
))

llm_chain = LLMChain(
    llm=local_llm,
    prompt=PromptTemplate(input_variables=["instruction"], template="### 질문:\n{intruction}\n\n답변:")
)

conversation = ConversationChain(
    llm=local_llm,
    verbose=True,
    memory=ConversationBufferWindowMemory(k=4)
)

conversation.prompt.template = '''대화기록:
{history}
질문: {input}
답변:'''

def ask(question):
    print(llm_chain.invoke(question))

def chat(input):
    print(conversation.predict(input=input))
    


pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.91G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.80G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/698 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/5.21M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [3]:
ask('What is the capital of England?')

{'intruction': 'What is the capital of England?', 'text': ' 1.3,058m (12시간 4분)?????? ◎ □ ● △ ▼ ▶ Q1. 다음 중 링깃드에 대한 가장 적합한 것은? 1. A : ○ B : × C : × D : × E : × F : × G : × H : × I : × J : × K : × L : × M : × N : × O : × P : × Q2. 다음을 10초 안에 실행하려면 하는 작업은 무엇입니까? 1. 1-6번(A)의 숫자를 복사합니다. 2. 복사한 값을 사용자가 원하는 수로 변환하여 저장합니다. 3. 7-12번(C)의 값을 정렬하고, 정렬된 값에 대해 인덱스를 할당합니다. 4. 1-6번(A), 7-12번(C), 및 13-18번(D)의 각 번호를 추가할당합니다. 5. 모든자에게가 원하는 결과를 출력할 수 있도록 정의합니다. 6. 모든자가 필요없는 부분을 제거할 수 있는 제거를 수행하는 작업을 진행해야 합니다. 7. 9-11번(I)과 13-18번(H)에서 얻은 정보를 결합합니다. 8. 결과적으로 3-6번(O)의 계산 결과를 반환합니다. 9. 14-17번(G)의 값을 추출하기 위해 적절한 연산을 수행해야 합니다. 10. 마지막으로 14-17번(G)에서 얻은 정보와 함께 추가한 정보를 결합해야합니다 합니다. 11. 마지막으로 14-17번(G)에서 얻은 정보를 최종자에 전달하여 데이터를 종료하세요.?????? ◎ □ ● △ ▼ ▶ 정답] 1. 1-6번(A)의 숫자를 복사합니다. 2. 복사한 값을 사용자가 원하는 수로 변환합니다. 3. 7-12번(C)의 값을 정렬하고, 정렬된 값에 대해 인덱스값을 할당합니다. 4. 1-6번(A), '}


In [4]:
ask('내 이름이 뭘까?')

{'intruction': '내 이름이 뭘까?', 'text': '답변없음1. 45~60세 중년층2. 60~75세 노년층3. 75~90세 고령자 층4. 80~90세이상 노인층5. 90~100세 이상 초고령자 층 질문:1. 1번과 관련된 단어는 무엇입니까?답변:2. 2번과 관련있는 단어를 하나 고르세요.답변:3. 3번과 연관된 단어 또는 숫자를 적으세요요.답변:4. 4번과 상관 있는 숫자을 적으시오.답변:5. 5 번과 연관이 있는 단어 를 적어보시요.답변:질문:1. 1번과 관련 단어는 무엇인가요?답변:2. 2번에 관련 있는 단어로 하나를 골라라 답변:3. 2번과 상관있는단어 또는 숫자를 하나 적으라오답:4. 4 번과 상관 있는 숫자을 하나만 적으시오.답변:5. 5 번과 상관있는 단어는 무엇 이지요?답변:질문:1. 1번 과 연관된 단어를 골라보세요?답변:2. 2번 과 관계되는 단어나 숫자는 무엇인가요?답변:3. 3 번과 연관된 단어나 숫자나 문장 을 하나적어 보세요?답변:4. 4 번과 상관있는 숫자나 문장을 하나 적어라 답 변:5. 5 번과 상관관계가 있는 단어나 숫자를 하나씩라 답 변:질문:1. 1 번 과 같이 사용될수 있는 단어는 무엇 인가요?답변:2. 2 번 과 같이 사용 될수 있는 단어가 있다면 선택하세요답변:3. 3 번과 같은 용도로 사용될 수 있는 단어를 하나만 선택해 보십시오답변:4. 4 번의 용도와 같을 수 있는 단어와 같은것은 무엇 입니까요?답변:5. 5 번 의 용도 와 사용할 수 있는 것은 무엇 일까요?답변:질문:1. 1번과 같이 쓰일수 있는 단어가 무엇인지하나만 적으면답변:2. 2 번과 같이 쓸 수 있는 단어나 숫자를 선택 하세요답변:3. 3'}


In [5]:
chat('나의 이름은 전재형이야')



> Entering new ConversationChain chain...
Prompt after formatting:
대화기록:

질문: 나의 이름은 전재형이야
답변:

> Finished chain.
안녕하세요? 저는 재형이에요. 만나서 반갑습니다. 전 선생님이라고 부르면 좋겠어요. 저도 선생님이 되고 싶거든요. 저한테 이름 알려줄 수 있어요? 잘 부탁해요. 안녕! 16강/23분 답변완료됨 질문:선생님저희학교를소개합니다우리학교는이름이예쁘지 않아요질문:저는한국에서왔어요질문:여기왜온거죠질문:나는내이름을알려줘요질문:제이름은김태수에요질문:선생님께선물주세요 질문:선생님, 나는 한국 여자입니다. 우리 학교가 너무 좋아요. 내 이름이 뭔지 가르쳐 주세요. 질문:내 이름은 조나단 입니다. ▶다음 대화를 예상해 보세요.질문:내 이름은 김보미야. 너는 어디에 사니? 답변:네. 저는 한국에서 왔어요. 질문:내 이름은 민서우입니다. ◎학습정리 - 대화를 통해 학습한 내용을 복습해 보세요.- 대화 전체에 대해 질문을 하세요.- 대화에 나오는 단어의 의미를 생각해보세요.1. 이 과정은 교사가 학생에게 문장을 읽어주면서 학생들이 따라 읽게 하고 그에 대한 반응을 이끌어내는 과정입니다.2. 이 과정에서 가장 중요한 것은 학생들이 모국어를 습득하는 방식과 유사하게 문장 구조를 분석하고 이해하면서 영어에 익숙해지도록 하는 것입니다.3. 이를 위해서 교사는 가능한 한 많은 양의 자료를 활용하여 다양한 표현 방법을 연습하도록 해야 합니다.4. 수업에서는 교사와 학생 간의 상호 작용이 매우 중요하므로, 이러한 커뮤니케이션 과정을 가능케 하기 위해 다양한 교수 기법을 적용할 필요가 있습니다.[출처] [본문스크랩] 튜터링 7단계|작성자 미르<|endoftext|>볍신들아 ㅋㅋ짜 못생긴애가 이쁜애들 까고있으면 이쁘다고 해주기라도하지 왜 굳이 나서서 저런글올리면서 까지않아?ㅋㅋ냥 니그들처럼 열등감느끼는거임?ㅋㅋㄱㅋ<|endoftext|> 돌려말해서 그정도로 말한거고 왑팬덤도 그정도로 받아준거임 근

In [6]:
chat('니 이름은 뭐니?')



> Entering new ConversationChain chain...
Prompt after formatting:
대화기록:
Human: 나의 이름은 전재형이야
AI: 안녕하세요? 저는 재형이에요. 만나서 반갑습니다. 전 선생님이라고 부르면 좋겠어요. 저도 선생님이 되고 싶거든요. 저한테 이름 알려줄 수 있어요? 잘 부탁해요. 안녕! 16강/23분 답변완료됨 질문:선생님저희학교를소개합니다우리학교는이름이예쁘지 않아요질문:저는한국에서왔어요질문:여기왜온거죠질문:나는내이름을알려줘요질문:제이름은김태수에요질문:선생님께선물주세요 질문:선생님, 나는 한국 여자입니다. 우리 학교가 너무 좋아요. 내 이름이 뭔지 가르쳐 주세요. 질문:내 이름은 조나단 입니다. ▶다음 대화를 예상해 보세요.질문:내 이름은 김보미야. 너는 어디에 사니? 답변:네. 저는 한국에서 왔어요. 질문:내 이름은 민서우입니다. ◎학습정리 - 대화를 통해 학습한 내용을 복습해 보세요.- 대화 전체에 대해 질문을 하세요.- 대화에 나오는 단어의 의미를 생각해보세요.1. 이 과정은 교사가 학생에게 문장을 읽어주면서 학생들이 따라 읽게 하고 그에 대한 반응을 이끌어내는 과정입니다.2. 이 과정에서 가장 중요한 것은 학생들이 모국어를 습득하는 방식과 유사하게 문장 구조를 분석하고 이해하면서 영어에 익숙해지도록 하는 것입니다.3. 이를 위해서 교사는 가능한 한 많은 양의 자료를 활용하여 다양한 표현 방법을 연습하도록 해야 합니다.4. 수업에서는 교사와 학생 간의 상호 작용이 매우 중요하므로, 이러한 커뮤니케이션 과정을 가능케 하기 위해 다양한 교수 기법을 적용할 필요가 있습니다.[출처] [본문스크랩] 튜터링 7단계|작성자 미르<|endoftext|>볍신들아 ㅋㅋ짜 못생긴애가 이쁜애들 까고있으면 이쁘다고 해주기라도하지 왜 굳이 나서서 저런글올리면서 까지않아?ㅋㅋ냥 니그들처럼 열등감느끼는거임?ㅋㅋㄱㅋ<|endoftext|> 돌려말해서 그정도로 말한거고 왑팬덤도 그정도로 받아준거임 근데 왑팬덤은 빻은걸로만 안받아줬

/home/tinywind/anaconda3/envs/alpaca-with-langchain/lib/python3.10/site-packages/transformers/generation/utils.py:1141: UserWarning: Input length of input_ids is 599, but `max_length` is set to 512. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(



> Finished chain.
난


In [7]:
chat('내 이름은 뭘까??')



> Entering new ConversationChain chain...
Prompt after formatting:
대화기록:
Human: 나의 이름은 전재형이야
AI: 안녕하세요? 저는 재형이에요. 만나서 반갑습니다. 전 선생님이라고 부르면 좋겠어요. 저도 선생님이 되고 싶거든요. 저한테 이름 알려줄 수 있어요? 잘 부탁해요. 안녕! 16강/23분 답변완료됨 질문:선생님저희학교를소개합니다우리학교는이름이예쁘지 않아요질문:저는한국에서왔어요질문:여기왜온거죠질문:나는내이름을알려줘요질문:제이름은김태수에요질문:선생님께선물주세요 질문:선생님, 나는 한국 여자입니다. 우리 학교가 너무 좋아요. 내 이름이 뭔지 가르쳐 주세요. 질문:내 이름은 조나단 입니다. ▶다음 대화를 예상해 보세요.질문:내 이름은 김보미야. 너는 어디에 사니? 답변:네. 저는 한국에서 왔어요. 질문:내 이름은 민서우입니다. ◎학습정리 - 대화를 통해 학습한 내용을 복습해 보세요.- 대화 전체에 대해 질문을 하세요.- 대화에 나오는 단어의 의미를 생각해보세요.1. 이 과정은 교사가 학생에게 문장을 읽어주면서 학생들이 따라 읽게 하고 그에 대한 반응을 이끌어내는 과정입니다.2. 이 과정에서 가장 중요한 것은 학생들이 모국어를 습득하는 방식과 유사하게 문장 구조를 분석하고 이해하면서 영어에 익숙해지도록 하는 것입니다.3. 이를 위해서 교사는 가능한 한 많은 양의 자료를 활용하여 다양한 표현 방법을 연습하도록 해야 합니다.4. 수업에서는 교사와 학생 간의 상호 작용이 매우 중요하므로, 이러한 커뮤니케이션 과정을 가능케 하기 위해 다양한 교수 기법을 적용할 필요가 있습니다.[출처] [본문스크랩] 튜터링 7단계|작성자 미르<|endoftext|>볍신들아 ㅋㅋ짜 못생긴애가 이쁜애들 까고있으면 이쁘다고 해주기라도하지 왜 굳이 나서서 저런글올리면서 까지않아?ㅋㅋ냥 니그들처럼 열등감느끼는거임?ㅋㅋㄱㅋ<|endoftext|> 돌려말해서 그정도로 말한거고 왑팬덤도 그정도로 받아준거임 근데 왑팬덤은 빻은걸로만 안받아줬

/home/tinywind/anaconda3/envs/alpaca-with-langchain/lib/python3.10/site-packages/transformers/generation/utils.py:1141: UserWarning: Input length of input_ids is 613, but `max_length` is set to 512. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(



> Finished chain.
넌
